# 第十章 数据聚合与分组运算

## 10.1 Groupby机制

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
....: 'key2' : ['one', 'two', 'one', 'two', 'one'],
....: 'data1' : np.random.randn(5),
....: 'data2' : np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,-0.249556,-0.371529
1,a,two,-1.410713,-1.148460
2,b,one,-1.927232,-1.397929
3,b,two,-1.923695,-1.047691
4,a,one,-1.326035,-0.933806


In [4]:
grouped = df['data1'].groupby(df['key1'])

In [5]:
grouped
#该变量实际上还没进行任何计算，只是含有了"对数据data1按照key1分组"的分组方法

In [6]:
grouped.mean()

key1
a   -0.995435
b   -1.925464
Name: data1, dtype: float64

In [7]:
grouped.sum()

key1
a   -2.986304
b   -3.850927
Name: data1, dtype: float64

In [8]:
grouped.count()

key1
a    3
b    2
Name: data1, dtype: int64

In [9]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
#传入多个分组列，会按照层次进行分组

In [10]:
means

key1  key2
a     one    -0.787796
      two    -1.410713
b     one    -1.927232
      two    -1.923695
Name: data1, dtype: float64

In [11]:
means2 = df['data2'].groupby([df['key1'], df['key2']]).mean()

In [12]:
means2
#计算ddata2的分组平均值
#如何同时计算两组数据的分组平均值？

key1  key2
a     one    -0.652667
      two    -1.148460
b     one    -1.397929
      two    -1.047691
Name: data2, dtype: float64

In [13]:
df.groupby([df['key1'], df['key2']]).mean()
#不对df指定索引，就会对除了指定的分组列之外的列计算分组后的平均值
#并且结果是DF类型的

data1     data2
key1 key2                    
a    one  -0.787796 -0.652667
     two  -1.410713 -1.148460
b    one  -1.927232 -1.397929
     two  -1.923695 -1.047691

In [14]:
means.unstack()#Series类型的结果，可以通过unstack函数转换为DF类型

key2,one,two
key1,,
a,-0.787796,-1.410713
b,-1.927232,-1.923695


In [15]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])

In [16]:
states

array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'], dtype='<U10')

In [17]:
years = np.array([2005, 2005, 2006, 2005, 2006])

In [18]:
years

array([2005, 2005, 2006, 2005, 2006])

In [19]:
df['data1'].groupby([states, years]).mean()
#使用长度适当的数组进行分组，类似于索引

California  2005   -1.410713
            2006   -1.927232
Ohio        2005   -1.086626
            2006   -1.326035
Name: data1, dtype: float64

In [20]:
df.groupby([states, years]).mean()
#不对df使用索引，会对data1,data2都进行计算
#这在DF不存在有意义的索引时会有用
#但难道不担心出错吗？

data1     data2
California 2005 -1.410713 -1.148460
           2006 -1.927232 -1.397929
Ohio       2005 -1.086626 -0.709610
           2006 -1.326035 -0.933806

In [21]:
df.groupby('key1').mean()
#将列名用作分组键

,data1,data2
key1,,
a,-0.995435,-0.817932
b,-1.925464,-1.222810


In [22]:
df.groupby(['key1', 'key2']).mean()
#按两列分组，实现效果类似于sql

data1     data2
key1 key2                    
a    one  -0.787796 -0.652667
     two  -1.410713 -1.148460
b    one  -1.927232 -1.397929
     two  -1.923695 -1.047691

In [23]:
df.groupby(['key1', 'key2']).size()
#size和count有区别吗？

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

In [24]:
df.groupby(['key1', 'key2']).count()

data1  data2
key1 key2              
a    one       2      2
     two       1      1
b    one       1      1
     two       1      1

In [25]:
df

,key1,key2,data1,data2
0,a,one,-0.249556,-0.371529
1,a,two,-1.410713,-1.148460
2,b,one,-1.927232,-1.397929
3,b,two,-1.923695,-1.047691
4,a,one,-1.326035,-0.933806


### 对分组进行迭代

In [26]:
#groupby对象是可迭代的，可以产生二元元组：分组名+数据块#分组名+均值
for name, group in df.groupby('key1'):
    print(name)
    print(group)
    print(group.mean()) #该行会打印出data1,data2的均值
    print(group.sum()) #该行会打印出data1,data2的和

a
  key1 key2     data1     data2
0    a  one -0.249556 -0.371529
1    a  two -1.410713 -1.148460
4    a  one -1.326035 -0.933806
data1   -0.995435
data2   -0.817932
dtype: float64
key1           aaa
key2     onetwoone
data1      -2.9863
data2     -2.45379
dtype: object
b
  key1 key2     data1     data2
2    b  one -1.927232 -1.397929
3    b  two -1.923695 -1.047691
data1   -1.925464
data2   -1.222810
dtype: float64
key1          bb
key2      onetwo
data1   -3.85093
data2   -2.44562
dtype: object


In [27]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)
#多重键分组，要用元素个数相同的元组进行迭代，结果是按多重键的元组对进行分类打印的

('a', 'one')
  key1 key2     data1     data2
0    a  one -0.249556 -0.371529
4    a  one -1.326035 -0.933806
('a', 'two')
  key1 key2     data1    data2
1    a  two -1.410713 -1.14846
('b', 'one')
  key1 key2     data1     data2
2    b  one -1.927232 -1.397929
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.923695 -1.047691


In [28]:
pieces = dict(list(df.groupby('key1')))

In [29]:
pieces['b']

,key1,key2,data1,data2
2,b,one,-1.927232,-1.397929
3,b,two,-1.923695,-1.047691


In [30]:
pieces

{'a':   key1 key2     data1     data2
 0    a  one -0.249556 -0.371529
 1    a  two -1.410713 -1.148460
 4    a  one -1.326035 -0.933806, 'b':   key1 key2     data1     data2
 2    b  one -1.927232 -1.397929
 3    b  two -1.923695 -1.047691}

In [31]:
df.groupby('key1')

In [32]:
list(df.groupby('key1'))

[('a',   key1 key2     data1     data2
  0    a  one -0.249556 -0.371529
  1    a  two -1.410713 -1.148460
  4    a  one -1.326035 -0.933806), ('b',   key1 key2     data1     data2
  2    b  one -1.927232 -1.397929
  3    b  two -1.923695 -1.047691)]

In [33]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [34]:
grouped = df.groupby(df.dtypes, axis=1)
#可以使用dtyeps对df的数据进行分组

In [35]:
grouped

In [36]:
for dtype, group in grouped:
    print(dtype)
    print(group)
    print(group.mean())#object对象没有均值，就会跳过去，不会报错
    print(group.sum()) #该行会打印出和

float64
      data1     data2
0 -0.249556 -0.371529
1 -1.410713 -1.148460
2 -1.927232 -1.397929
3 -1.923695 -1.047691
4 -1.326035 -0.933806
data1   -1.367446
data2   -0.979883
dtype: float64
data1   -6.837231
data2   -4.899415
dtype: float64
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one
Series([], dtype: float64)
key1              aabba
key2    onetwoonetwoone
dtype: object


### 选取一列或列的子集

In [37]:
df['data1'].groupby(df['key1'])
#对df的data1列按照key1分组

In [38]:
#语法糖
df.groupby('key1')['data1']

In [39]:
df['data1'].groupby(df['key1']).describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-0.995435,0.647336,-1.410713,-1.368374,-1.326035,-0.787796,-0.249556
b,2.0,-1.925464,0.002501,-1.927232,-1.926348,-1.925464,-1.924579,-1.923695


In [40]:
df.groupby('key1')['data1'].describe()

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-0.995435,0.647336,-1.410713,-1.368374,-1.326035,-0.787796,-0.249556
b,2.0,-1.925464,0.002501,-1.927232,-1.926348,-1.925464,-1.924579,-1.923695


In [41]:
df.groupby('key1')[['data2']].describe()

data2                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.817932  0.401217 -1.148460 -1.041133 -0.933806 -0.652667   
b      2.0 -1.222810  0.247656 -1.397929 -1.310370 -1.222810 -1.135250   

                
           max  
key1            
a    -0.371529  
b    -1.047691

In [42]:
df.groupby('key1')['data2'].describe()
#与上式的区别只是没有了data2的标题

,count,mean,std,min,25%,50%,75%,max
key1,,,,,,,,
a,3.0,-0.817932,0.401217,-1.148460,-1.041133,-0.933806,-0.652667,-0.371529
b,2.0,-1.222810,0.247656,-1.397929,-1.310370,-1.222810,-1.135250,-1.047691


In [43]:
df[['data2']].groupby(df['key1']).describe()
#正常的写法是这样的

data2                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.817932  0.401217 -1.148460 -1.041133 -0.933806 -0.652667   
b      2.0 -1.222810  0.247656 -1.397929 -1.310370 -1.222810 -1.135250   

                
           max  
key1            
a    -0.371529  
b    -1.047691

In [44]:
df.groupby('key1')['data1','data2'].describe()
#

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.995435  0.647336 -1.410713 -1.368374 -1.326035 -0.787796   
b      2.0 -1.925464  0.002501 -1.927232 -1.926348 -1.925464 -1.924579   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.249556   3.0 -0.817932  0.401217 -1.148460 -1.041133 -0.933806   
b    -1.923695   2.0 -1.222810  0.247656 -1.397929 -1.310370 -1.222810   

                          
           75%       max  
key1                      
a    -0.652667 -0.371529  
b    -1.135250 -1.047691

In [45]:
df.groupby(df['key1']).describe()
#如果想在df后加索引，实现同样的效果，该怎么加？
#貌似这是python基础语法的问题。。。

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.995435  0.647336 -1.410713 -1.368374 -1.326035 -0.787796   
b      2.0 -1.925464  0.002501 -1.927232 -1.926348 -1.925464 -1.924579   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.249556   3.0 -0.817932  0.401217 -1.148460 -1.041133 -0.933806   
b    -1.923695   2.0 -1.222810  0.247656 -1.397929 -1.310370 -1.222810   

                          
           75%       max  
key1                      
a    -0.652667 -0.371529  
b    -1.135250 -1.047691

In [46]:
df.groupby(['key1', 'key2'])[['data2']].mean()
#对data2的数据，按照key1，key2列分组求均值

data2
key1 key2          
a    one  -0.652667
     two  -1.148460
b    one  -1.397929
     two  -1.047691

In [47]:
#不用语法糖该怎么写？

In [48]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
#索引操作
#类似于grouped = df['data1'].groupby(df['key1'])
#该变量实际上还没进行任何计算，只是含有了"对数据data2按照key1，key2分组"的分组方法

In [49]:
s_grouped

In [50]:
s_grouped.mean()

key1  key2
a     one    -0.652667
      two    -1.148460
b     one    -1.397929
      two    -1.047691
Name: data2, dtype: float64

### 通过字段或者Series进行分组

In [51]:
#除了数组外，海可以用其他形式的分组信息
people = pd.DataFrame(np.random.randn(5, 5),
                      columns=['a', 'b', 'c', 'd', 'e'],
                      index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])

In [52]:
people

,a,b,c,d,e
Joe,0.641856,0.282004,1.038634,-0.846908,2.182948
Steve,-1.336201,-0.655686,0.615457,0.253140,0.332245
Wes,-1.201985,0.945739,0.072285,-0.694589,0.868901
Jim,1.551461,0.513565,-1.073324,0.419274,-0.694358
Travis,2.120025,0.249518,-0.695867,0.476079,-0.409290


In [53]:
people.iloc[2:3, [1, 2]] = np.nan 
# 对第三行的第二、三列赋缺失值

In [54]:
people

,a,b,c,d,e
Joe,0.641856,0.282004,1.038634,-0.846908,2.182948
Steve,-1.336201,-0.655686,0.615457,0.253140,0.332245
Wes,-1.201985,NaN,NaN,-0.694589,0.868901
Jim,1.551461,0.513565,-1.073324,0.419274,-0.694358
Travis,2.120025,0.249518,-0.695867,0.476079,-0.409290


In [55]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue','d': 'blue', 'e': 'red', 'f' : 'orange'}
#使用字典表示分组关系，注意是对列名进行分组

In [56]:
by_column = people.groupby(mapping, axis=1)
#使用mappiing进行分组，注意是以列为轴，也就是按行分组求和

In [57]:
by_column.sum()#注意结果没有缺失值，因为存在缺失值的b，c列并不在一组（'b': 'red', 'c': 'blue'）

,blue,red
Joe,0.191726,3.106808
Steve,0.868596,-1.659642
Wes,-0.694589,-0.333085
Jim,-0.654050,1.370669
Travis,-0.219787,1.960253


In [58]:
map_series = pd.Series(mapping)#除了字典，Series对象也可传递给groupby来进行分组

In [59]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [60]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 通过函数进行分组

In [61]:
people.groupby(len).sum()
#使用len函数计算索引列（也就是姓名）的长度，然后进行分组
#如果两个姓名长度一致，那就会被分为一组

,a,b,c,d,e
3,0.991332,0.795569,-0.034690,-1.122223,2.357491
5,-1.336201,-0.655686,0.615457,0.253140,0.332245
6,2.120025,0.249518,-0.695867,0.476079,-0.409290


In [62]:
people.groupby(len).size()

3    3
5    1
6    1
dtype: int64

In [63]:
people.groupby(len).count()
#索引为3的那一行，因为b,c列为缺失值，因此是2

,a,b,c,d,e
3,3,2,2,3,3
5,1,1,1,1,1
6,1,1,1,1,1


In [64]:
key_list = ['one', 'one', 'one', 'two', 'two']

In [65]:
people.groupby([len, key_list]).min()
#将函数和数组、列表、字典、Series混合使用，进行分组也可以
#分组时，传递进去的都会被转为数组

a         b         c         d         e
3 one -1.201985  0.282004  1.038634 -0.846908  0.868901
  two  1.551461  0.513565 -1.073324  0.419274 -0.694358
5 one -1.336201 -0.655686  0.615457  0.253140  0.332245
6 two  2.120025  0.249518 -0.695867  0.476079 -0.409290

### 根据索引级别分组

In [66]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],[1, 3, 5, 1, 3]],names=['cty', 'tenor'])
#使用pandas的MultiIndex.from_array方法定义一个层次化索引

In [67]:
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
#定义一个层次化索引的DF，列索引有两层

In [68]:
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0     -0.548996  0.409008 -0.666209 -1.371361  0.652576
1     -1.308530  0.868955  1.866993 -0.626162  1.790991
2      0.103255 -0.412884 -0.891330 -0.387188 -2.541828
3     -1.825834 -1.084506 -0.098265 -0.422299 -0.659076

In [69]:
hier_df.groupby(level='cty', axis=1).count()
#根据索引级别分组,使用索引级别的名称

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [70]:
hier_df.groupby(level=0, axis=1).count()
#根据索引级别分组,使用索引级别的序号，0位第一层，与上述结果一直

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


In [71]:
hier_df.groupby(level=0, axis=0).count()
#根据索引级别分组,使用索引级别的序号，1为次级索引

cty   US       JP   
tenor  1  3  5  1  3
0      1  1  1  1  1
1      1  1  1  1  1
2      1  1  1  1  1
3      1  1  1  1  1

## 10.2 数据聚合

In [72]:
df

,key1,key2,data1,data2
0,a,one,-0.249556,-0.371529
1,a,two,-1.410713,-1.148460
2,b,one,-1.927232,-1.397929
3,b,two,-1.923695,-1.047691
4,a,one,-1.326035,-0.933806


In [73]:
grouped = df.groupby('key1')
#按照key1进行groupby

In [74]:
grouped

In [75]:
grouped['data1'].quantile(0.5)
#计算data1按照key1分组后的百分位数

key1
a   -1.326035
b   -1.925464
Name: data1, dtype: float64

In [76]:
grouped['data1'].quantile(0.25)
#怎么同时返回多个百分位数？

key1
a   -1.368374
b   -1.926348
Name: data1, dtype: float64

In [77]:
#自定义聚合函数
def peak_to_peak(arr):
    return arr.max()-arr.min()
#对给定的array，返回最大值和最小值之差

In [78]:
grouped.agg(peak_to_peak)
#agg方法是在指定轴上聚合，默认是列方向聚合？

,data1,data2
key1,,
a,1.161156,0.776931
b,0.003537,0.350238


In [79]:
grouped.agg??

In [80]:
grouped.describe()
#注意grouped是定义为按照key1进行groupby的

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0 -0.995435  0.647336 -1.410713 -1.368374 -1.326035 -0.787796   
b      2.0 -1.925464  0.002501 -1.927232 -1.926348 -1.925464 -1.924579   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a    -0.249556   3.0 -0.817932  0.401217 -1.148460 -1.041133 -0.933806   
b    -1.923695   2.0 -1.222810  0.247656 -1.397929 -1.310370 -1.222810   

                          
           75%       max  
key1                      
a    -0.652667 -0.371529  
b    -1.135250 -1.047691

### 面向列的多函数应用

In [81]:
tips = pd.read_csv('examples/tips.csv')

In [82]:
tips.head()

,total_bill,tip,smoker,day,time,size
0,16.99,1.01,No,Sun,Dinner,2
1,10.34,1.66,No,Sun,Dinner,3
2,21.01,3.50,No,Sun,Dinner,3
3,23.68,3.31,No,Sun,Dinner,2
4,24.59,3.61,No,Sun,Dinner,4


In [83]:
tips['tip_pct']=tips['tip']/tips['total_bill']
#定义tip_pct,一步实现了新增列和填充数据

In [84]:
tips.head()

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808


In [85]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


In [86]:
grouped=tips.groupby(['day','smoker'])

In [90]:
grouped

In [88]:
grouped_pct=grouped['tip_pct']

In [89]:
grouped_pct.agg('mean')

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [91]:
grouped_pct.agg(['mean', 'std', peak_to_peak])
#传入一组函数，得到DF会以该函数名命名，这个理解起来很直观

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

In [94]:
grouped_pct.agg([('均值','mean'), ('标准差','std'), peak_to_peak])
#当然也可以对列进行重命名
#重命名要用元组列表传递，元组内，想要重名名的名称放在前边，函数名称放在后边——感觉很不自然

均值       标准差  <function peak_to_peak at 0x0000000008EF59D8>
day  smoker                                                                   
Fri  No      0.151650  0.028123                                       0.067349
     Yes     0.174783  0.051293                                       0.159925
Sat  No      0.158048  0.039767                                       0.235193
     Yes     0.147906  0.061375                                       0.290095
Sun  No      0.160113  0.042347                                       0.193226
     Yes     0.187250  0.154134                                       0.644685
Thur No      0.160298  0.038774                                       0.193350
     Yes     0.163863  0.039389                                       0.151240

In [95]:
functions = ['count', 'mean', 'max']
#定义一个由函数名组成的列表

In [96]:
result = grouped['tip_pct', 'total_bill'].agg(functions)
#然后就可以对DF内指定的列同时应用上述列表所包含的函数了

In [97]:
result
#结果有层次化的列

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [98]:
result['tip_pct']
#只取第一层的tip_pct这一列

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [99]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]

In [100]:
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

In [101]:
 grouped.agg({'tip' : np.max, 'size' : 'sum'})
#传入一个列名到函数名的字典，可以对不同的列分别使用不同的函数
#得到的结果，会用原来的列名命名新的DF的列——这不科学

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [108]:
 grouped.agg(np.max)#['tip']

total_bill    tip    time  size   tip_pct
day  smoker                                           
Fri  No           22.75   3.50   Lunch     3  0.187735
     Yes          40.17   4.73   Lunch     4  0.263480
Sat  No           48.33   9.00  Dinner     4  0.291990
     Yes          50.81  10.00  Dinner     5  0.325733
Sun  No           48.17   6.00  Dinner     6  0.252672
     Yes          45.35   6.50  Dinner     5  0.710345
Thur No           41.19   6.70   Lunch     6  0.266312
     Yes          43.11   5.00   Lunch     4  0.241255

In [109]:
 grouped.agg('sum')#['size']

total_bill     tip  size   tip_pct
day  smoker                                    
Fri  No           73.68   11.25     9  0.606602
     Yes         252.20   40.71    31  2.621746
Sat  No          884.78  139.63   115  7.112145
     Yes         893.62  120.77   104  6.212055
Sun  No         1168.88  180.57   167  9.126438
     Yes         458.28   66.82    49  3.557756
Thur No          770.09  120.32   112  7.213414
     Yes         326.24   51.51    40  2.785676

In [110]:
grouped.agg({'tip_pct' : ['min', 'max', 'mean', 'std'], 'size' : 'sum'})
#如果传入的字典的值是个列表，则创建的DF会有分层列索引
#第二次列索引使用函数名命名

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 以“没有行索引”的形式返回聚合数据

In [111]:
tips.groupby(['day', 'smoker'], as_index=False).mean()
#使用as_index=False 取消掉默认的行索引，使之成为列

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


In [112]:
tips.groupby??
#
#tips.groupby(
#    by=None,
#    axis=0,
#    level=None,
#    as_index=True,
#    sort=True,
#    group_keys=True,
#    squeeze=False,
#    observed=False,
#    **kwargs,
#)

## 10.3 apply:一般性的“拆分—应用—合并”

In [113]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by=column)[-n:]

In [114]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [115]:
top(tips)#默认n=5

,total_bill,tip,smoker,day,time,size,tip_pct
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
232,11.61,3.39,No,Sat,Dinner,2,0.291990
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [116]:
tips.groupby('smoker').apply(top)
#使用apply，可以更加方便地使用自定义函数

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

In [117]:
tips.groupby('smoker')

In [119]:
tips.groupby('smoker').apply(max)

,total_bill,tip,smoker,day,time,size,tip_pct
smoker,,,,,,,
No,48.33,9.0,No,Thur,Lunch,6,0.291990
Yes,50.81,10.0,Yes,Thur,Lunch,5,0.710345


In [120]:
tips.groupby(['smoker', 'day']).apply(top, n=1, column='total_bill')

total_bill    tip smoker   day    time  size   tip_pct
smoker day                                                             
No     Fri  94        22.75   3.25     No   Fri  Dinner     2  0.142857
       Sat  212       48.33   9.00     No   Sat  Dinner     4  0.186220
       Sun  156       48.17   5.00     No   Sun  Dinner     6  0.103799
       Thur 142       41.19   5.00     No  Thur   Lunch     5  0.121389
Yes    Fri  95        40.17   4.73    Yes   Fri  Dinner     4  0.117750
       Sat  170       50.81  10.00    Yes   Sat  Dinner     3  0.196812
       Sun  182       45.35   3.50    Yes   Sun  Dinner     3  0.077178
       Thur 197       43.11   5.00    Yes  Thur   Lunch     4  0.115982

In [121]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [122]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,0.159328,0.039910,0.056797,0.136906,0.155625,0.185014,0.291990
Yes,93.0,0.163196,0.085119,0.035638,0.106771,0.153846,0.195059,0.710345


In [123]:
result.unstack('smoker')

       smoker
count  No        151.000000
       Yes        93.000000
mean   No          0.159328
       Yes         0.163196
std    No          0.039910
       Yes         0.085119
min    No          0.056797
       Yes         0.035638
25%    No          0.136906
       Yes         0.106771
50%    No          0.155625
       Yes         0.153846
75%    No          0.185014
       Yes         0.195059
max    No          0.291990
       Yes         0.710345
dtype: float64

### 禁止分组键

In [124]:
tips.groupby('smoker', group_keys=False).apply(top)
#禁止分组键group_keys=False时，分组键会被禁止作为层次化索引

,total_bill,tip,smoker,day,time,size,tip_pct
88,24.71,5.85,No,Thur,Lunch,2,0.236746
185,20.69,5.00,No,Sun,Dinner,5,0.241663
51,10.29,2.60,No,Sun,Dinner,2,0.252672
149,7.51,2.00,No,Thur,Lunch,2,0.266312
232,11.61,3.39,No,Sat,Dinner,2,0.291990
109,14.31,4.00,Yes,Sat,Dinner,2,0.279525
183,23.17,6.50,Yes,Sun,Dinner,4,0.280535
67,3.07,1.00,Yes,Sat,Dinner,1,0.325733
178,9.60,4.00,Yes,Sun,Dinner,2,0.416667
172,7.25,5.15,Yes,Sun,Dinner,2,0.710345


In [125]:
tips.groupby('smoker').apply(top)
#作为对比，分组键默认会跟原始索引构成层次化索引

total_bill   tip smoker   day    time  size   tip_pct
smoker                                                           
No     88        24.71  5.85     No  Thur   Lunch     2  0.236746
       185       20.69  5.00     No   Sun  Dinner     5  0.241663
       51        10.29  2.60     No   Sun  Dinner     2  0.252672
       149        7.51  2.00     No  Thur   Lunch     2  0.266312
       232       11.61  3.39     No   Sat  Dinner     2  0.291990
Yes    109       14.31  4.00    Yes   Sat  Dinner     2  0.279525
       183       23.17  6.50    Yes   Sun  Dinner     4  0.280535
       67         3.07  1.00    Yes   Sat  Dinner     1  0.325733
       178        9.60  4.00    Yes   Sun  Dinner     2  0.416667
       172        7.25  5.15    Yes   Sun  Dinner     2  0.710345

### 分位数和桶分析

In [126]:
frame = pd.DataFrame({'data1': np.random.randn(1000),'data2': np.random.randn(1000)})

In [127]:
quartiles = pd.cut(frame.data1, 4)
#使用cut函数将数据装入长度相等的桶中（即将最大书和最小数构成的区间，均分为四份，用来替代实际数据）

In [131]:
quartiles[:10]

0    (-2.701, -1.119]
1      (0.458, 2.034]
2     (-1.119, 0.458]
3     (-1.119, 0.458]
4     (-1.119, 0.458]
5     (-1.119, 0.458]
6      (0.458, 2.034]
7      (0.458, 2.034]
8     (-1.119, 0.458]
9     (-1.119, 0.458]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.701, -1.119] < (-1.119, 0.458] < (0.458, 2.034] < (2.034, 3.61]]

In [129]:
frame.head()

,data1,data2
0,-1.792049,-0.158284
1,1.646633,0.788937
2,-0.302591,-0.492878
3,-0.057211,-0.075771
4,0.219508,-0.044576


In [132]:
def get_stats(group):
    return {'min': group.min(), 'max': group.max(),'count': group.count(), 'mean': group.mean()}
#自定义一个统计函数，返回最小最大值，总数，均值

In [133]:
grouped = frame.data2.groupby(quartiles)
#对df的data2列按照所归属的区间进行分组——这么做可以把连续数据离散化

In [134]:
grouped.apply(get_stats).unstack()
#使用自定义的统计函数获取相应分组内的data1的最大最小值，数量，均值

,count,max,mean,min
data1,,,,
"(-2.701, -1.119]",142.0,2.521084,-0.005975,-2.758905
"(-1.119, 0.458]",540.0,3.281810,-0.026566,-2.889079
"(0.458, 2.034]",295.0,2.903823,-0.099917,-2.919676
"(2.034, 3.61]",23.0,2.378370,0.275905,-2.507237


In [135]:
grouped.apply(get_stats)

data1                  
(-2.701, -1.119]  count    142.000000
                  max        2.521084
                  mean      -0.005975
                  min       -2.758905
(-1.119, 0.458]   count    540.000000
                  max        3.281810
                  mean      -0.026566
                  min       -2.889079
(0.458, 2.034]    count    295.000000
                  max        2.903823
                  mean      -0.099917
                  min       -2.919676
(2.034, 3.61]     count     23.000000
                  max        2.378370
                  mean       0.275905
                  min       -2.507237
Name: data2, dtype: float64

In [142]:
grouping = pd.qcut(frame.data1, 10, labels=False)
#使用qcut，获得按分位数分组的结果
#labels=False表示不使用labels作为层次化索引

In [143]:
grouped = frame.data2.groupby(grouping)

In [144]:
grouped.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
0,100.0,2.521084,0.026250,-2.758905
1,100.0,2.243090,-0.128730,-2.079013
2,100.0,3.281810,0.053669,-2.193536
3,100.0,2.326764,0.035081,-2.436087
4,100.0,2.421899,-0.195122,-2.889079
5,100.0,2.180910,0.108729,-2.185843
6,100.0,2.099506,-0.075552,-2.631888
7,100.0,2.296838,-0.127908,-2.209375
8,100.0,2.276378,-0.053270,-2.919676


In [145]:
#作为对比，如果不指定labels=False，会是这样
grouping1 = pd.qcut(frame.data1, 10)

In [148]:
grouped1 = frame.data2.groupby(grouping1)

In [149]:
grouped1.apply(get_stats).unstack()

,count,max,mean,min
data1,,,,
"(-2.6959999999999997, -1.29]",100.0,2.521084,0.026250,-2.758905
"(-1.29, -0.895]",100.0,2.243090,-0.128730,-2.079013
"(-0.895, -0.556]",100.0,3.281810,0.053669,-2.193536
"(-0.556, -0.299]",100.0,2.326764,0.035081,-2.436087
"(-0.299, 0.00767]",100.0,2.421899,-0.195122,-2.889079
"(0.00767, 0.239]",100.0,2.180910,0.108729,-2.185843
"(0.239, 0.512]",100.0,2.099506,-0.075552,-2.631888
"(0.512, 0.794]",100.0,2.296838,-0.127908,-2.209375
"(0.794, 1.222]",100.0,2.276378,-0.053270,-2.919676


### 示例：用特定于分组的值填充缺失值

In [150]:
s = pd.Series(np.random.randn(6))

In [151]:
s[::2] = np.nan

In [152]:
s

0         NaN
1    0.450199
2         NaN
3   -1.936615
4         NaN
5    0.669137
dtype: float64

In [153]:
s.fillna(s.mean())

0   -0.272426
1    0.450199
2   -0.272426
3   -1.936615
4   -0.272426
5    0.669137
dtype: float64

In [165]:
states = ['Ohio', 'New York', 'Vermont', 'Florida', 'Oregon', 'Nevada', 'California', 'Idaho']

In [166]:
group_key = ['East'] * 4 + ['West'] * 4

In [167]:
data = pd.Series(np.random.randn(8), index=states)

In [168]:
data

Ohio          0.678933
New York      1.734982
Vermont      -0.338266
Florida      -0.560379
Oregon        0.802820
Nevada        0.618786
California   -0.204419
Idaho         1.084436
dtype: float64

In [169]:
data.groupby(group_key).mean()

East    0.378818
West    0.575406
dtype: float64

In [170]:
data[['Vermont', 'Nevada', 'Idaho']] = np.nan

In [171]:
data

Ohio          0.678933
New York      1.734982
Vermont            NaN
Florida      -0.560379
Oregon        0.802820
Nevada             NaN
California   -0.204419
Idaho              NaN
dtype: float64

In [172]:
data.groupby(group_key).mean()

East    0.617846
West    0.299200
dtype: float64

In [175]:
fill_mean = lambda g: g.fillna(g.mean())
#使用均值填充

In [176]:
data.groupby(group_key).apply(fill_mean)

Ohio          0.678933
New York      1.734982
Vermont       0.617846
Florida      -0.560379
Oregon        0.802820
Nevada        0.299200
California   -0.204419
Idaho         0.299200
dtype: float64

In [177]:
data.groupby(group_key).apply(fill_mean).groupby(group_key).mean()
#使用均值填充后，均值并不会变，但数据中没有缺失值了

East    0.617846
West    0.299200
dtype: float64

In [178]:
fill_values = {'East': 0.5, 'West': -1}
#使用字典预定义填充缺失值

In [180]:
fill_func = lambda g: g.fillna(fill_values[g.name])
#定义一个函数，使用预定义的字典填充缺失值

In [181]:
data.groupby(group_key).apply(fill_func)
#注意原来缺失值的地方分别按照字典进行了填充
#填充缺失值，技术上并不存在问题，关键是业务上决定该怎么填充

Ohio          0.678933
New York      1.734982
Vermont       0.500000
Florida      -0.560379
Oregon        0.802820
Nevada       -1.000000
California   -0.204419
Idaho        -1.000000
dtype: float64

### 示例：随机采样和排列

In [182]:
suits = ['H', 'S', 'C', 'D']
card_val = (list(range(1, 11)) + [10] * 3) * 4
base_names = ['A'] + list(range(2, 11)) + ['J', 'K', 'Q']
cards = []
for suit in ['H', 'S', 'C', 'D']:
    cards.extend(str(num) + suit for num in base_names)
deck = pd.Series(card_val, index=cards)

In [183]:
deck[:13]

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
dtype: int64

In [189]:
card_val

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 10,
 10,
 10]

In [190]:
base_names

['A', 2, 3, 4, 5, 6, 7, 8, 9, 10, 'J', 'K', 'Q']

In [191]:
cards

['AH',
 '2H',
 '3H',
 '4H',
 '5H',
 '6H',
 '7H',
 '8H',
 '9H',
 '10H',
 'JH',
 'KH',
 'QH',
 'AS',
 '2S',
 '3S',
 '4S',
 '5S',
 '6S',
 '7S',
 '8S',
 '9S',
 '10S',
 'JS',
 'KS',
 'QS',
 'AC',
 '2C',
 '3C',
 '4C',
 '5C',
 '6C',
 '7C',
 '8C',
 '9C',
 '10C',
 'JC',
 'KC',
 'QC',
 'AD',
 '2D',
 '3D',
 '4D',
 '5D',
 '6D',
 '7D',
 '8D',
 '9D',
 '10D',
 'JD',
 'KD',
 'QD']

In [193]:
deck

AH      1
2H      2
3H      3
4H      4
5H      5
6H      6
7H      7
8H      8
9H      9
10H    10
JH     10
KH     10
QH     10
AS      1
2S      2
3S      3
4S      4
5S      5
6S      6
7S      7
8S      8
9S      9
10S    10
JS     10
KS     10
QS     10
AC      1
2C      2
3C      3
4C      4
5C      5
6C      6
7C      7
8C      8
9C      9
10C    10
JC     10
KC     10
QC     10
AD      1
2D      2
3D      3
4D      4
5D      5
6D      6
7D      7
8D      8
9D      9
10D    10
JD     10
KD     10
QD     10
dtype: int64

In [184]:
#从整副牌中抽取五张的方法
def draw(deck, n=5):
    return deck.sample(n)#sample方法从总体中随机抽取n张牌

In [192]:
draw(deck)

AS     1
AD     1
4D     4
QS    10
4H     4
dtype: int64

In [196]:
draw(deck,n=3)#可以指定抽取的牌的数量

AC    1
4D    4
6H    6
dtype: int64

In [186]:
get_suit = lambda card: card[-1]#获取花色的方法

In [187]:
deck.groupby(get_suit).apply(draw, n=2)
#groupby语句按花色分组，然后apply抽牌方法，指定抽两张

C  2C      2
   5C      5
D  5D      5
   10D    10
H  JH     10
   4H      4
S  QS     10
   6S      6
dtype: int64

In [188]:
deck.groupby(get_suit, group_keys=False).apply(draw, n=2)
#取消按花色分组的层次索引，默认是开启的

9C      9
JC     10
4D      4
10D    10
3H      3
KH     10
KS     10
6S      6
dtype: int64

### 示例：分组加权平均数和相关系数

In [197]:
df = pd.DataFrame({'category': ['a', 'a', 'a', 'a','b', 'b', 'b', 'b'], 
                   'data': np.random.randn(8),
                   'weights': np.random.rand(8)})

In [198]:
df

,category,data,weights
0,a,0.008356,0.398483
1,a,-1.833178,0.096834
2,a,-2.660949,0.908175
3,a,0.888503,0.205277
4,b,1.557597,0.426154
5,b,0.538421,0.423167
6,b,-0.219944,0.726205
7,b,-0.846285,0.599360


In [199]:
grouped = df.groupby('category')

In [200]:
get_wavg = lambda g: np.average(g['data'], weights=g['weights'])
#lambda函数调用了np.average方法，传递值和权重的arrary，计算出加权平均值

In [211]:
np.average??
#np.average(a, axis=None, weights=None, returned=False)
#还可以指定轴

In [201]:
grouped.apply(get_wavg)
#对按照category分组的结果apply自定义的get_wavg方法，获取每个category的加权平均值

category
a   -1.497047
b    0.103299
dtype: float64

In [202]:
close_px = pd.read_csv('examples/stock_px_2.csv', parse_dates=True,index_col=0)

In [203]:
close_px.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2214 entries, 2003-01-02 to 2011-10-14
Data columns (total 4 columns):
AAPL    2214 non-null float64
MSFT    2214 non-null float64
XOM     2214 non-null float64
SPX     2214 non-null float64
dtypes: float64(4)
memory usage: 86.5 KB


In [204]:
close_px[-4:]

,AAPL,MSFT,XOM,SPX
2011-10-11,400.29,27.00,76.27,1195.54
2011-10-12,402.19,26.96,77.16,1207.25
2011-10-13,408.43,27.18,76.37,1203.66
2011-10-14,422.00,27.27,78.11,1224.58


In [217]:
close_px[:11]

,AAPL,MSFT,XOM,SPX
2003-01-02,7.40,21.11,29.22,909.03
2003-01-03,7.45,21.14,29.24,908.59
2003-01-06,7.45,21.52,29.96,929.01
2003-01-07,7.43,21.93,28.95,922.93
2003-01-08,7.28,21.31,28.83,909.93
2003-01-09,7.34,21.93,29.44,927.57
2003-01-10,7.36,21.97,29.03,927.57
2003-01-13,7.32,22.16,28.91,926.26
2003-01-14,7.30,22.39,29.17,931.66
2003-01-15,7.22,22.11,28.77,918.22


In [205]:
spx_corr = lambda x: x.corrwith(x['SPX'])
#自定义lambada函数，计算与spx（标准普尔500指数）的相关系数

In [220]:
close_px.corrwith??
#计算相关系数的方法
#close_px.corrwith(other, axis=0, drop=False, method='pearson')

In [206]:
rets = close_px.pct_change().dropna()
#计算各列的逐日变化率
#去掉缺失值所在的行

In [222]:
close_px.pct_change().head()
#不去掉缺失值，第一行由于没有办法和前一天的对比，肯定是缺失值

,AAPL,MSFT,XOM,SPX
2003-01-02,NaN,NaN,NaN,NaN
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086


In [216]:
rets[:10]

,AAPL,MSFT,XOM,SPX
2003-01-03,0.006757,0.001421,0.000684,-0.000484
2003-01-06,0.000000,0.017975,0.024624,0.022474
2003-01-07,-0.002685,0.019052,-0.033712,-0.006545
2003-01-08,-0.020188,-0.028272,-0.004145,-0.014086
2003-01-09,0.008242,0.029094,0.021159,0.019386
2003-01-10,0.002725,0.001824,-0.013927,0.000000
2003-01-13,-0.005435,0.008648,-0.004134,-0.001412
2003-01-14,-0.002732,0.010379,0.008993,0.005830
2003-01-15,-0.010959,-0.012506,-0.013713,-0.014426
2003-01-16,0.012465,-0.016282,0.004519,-0.003942


In [213]:
close_px.pct_change??
#计算变化的百分比的方法
#close_px.pct_change(
#    periods=1,
#    fill_method='pad',
#    limit=None,
#    freq=None,
#    **kwargs,
#)

In [207]:
get_year = lambda x: x.year
#定义一个从日期字段获取年份的方法

In [208]:
by_year = rets.groupby(get_year)
#按年对逐日变化率分组

In [214]:
by_year

In [209]:
by_year.apply(spx_corr)
#计算按年分组后的日均净利润变化率与spx的相关系数？
#为什么SPX的相关系数会是1.0？因为对SPX这一列，计算的是它和自己的相关系数
#对于前边几列，分别计算的是各列在不同年份与SPX的相关系数——可见相关度是非常强的

,AAPL,MSFT,XOM,SPX
2003,0.541124,0.745174,0.661265,1.0
2004,0.374283,0.588531,0.557742,1.0
2005,0.467540,0.562374,0.631010,1.0
2006,0.428267,0.406126,0.518514,1.0
2007,0.508118,0.658770,0.786264,1.0
2008,0.681434,0.804626,0.828303,1.0
2009,0.707103,0.654902,0.797921,1.0
2010,0.710105,0.730118,0.839057,1.0
2011,0.691931,0.800996,0.859975,1.0


In [210]:
by_year.apply(lambda g: g['AAPL'].corr(g['MSFT']))
#只计算按年份分组后AAPL与MSFT的日均利润变化率的相关系数——相关度逐年变化

2003    0.480868
2004    0.259024
2005    0.300093
2006    0.161735
2007    0.417738
2008    0.611901
2009    0.432738
2010    0.571946
2011    0.581987
dtype: float64

### 示例：组级别的线性回归

In [223]:
import statsmodels.api as sm
#使用statmodels.api计量经济学库，对各数据块执行普通最小二乘法

In [242]:
def regress(data, yvar, xvars):
    Y = data[yvar]
    X = data[xvars]
    X['intercept'] = 1.
    result = sm.OLS(Y, X).fit()
    return result.params
#上述定义的函数是什么意思？——使用普通最小二乘法计算data[yvar]对data[xvars]的线性回归
#X和Y定义了两个列，分别是data的[yvar]和[xvar]索引所对应的列
#X['intercept'] = 1 截距=1，是什么意思？
#调用sm包的OLS方法

In [243]:
sm.OLS??

In [244]:
by_year['AAPL']

In [245]:
by_year.apply(regress, 'AAPL', ['SPX'])
#计算AAPL对SPX收益率的线性回归

,SPX,intercept
2003,1.195406,0.000710
2004,1.363463,0.004201
2005,1.766415,0.003246
2006,1.645496,0.000080
2007,1.198761,0.003438
2008,0.968016,-0.001110
2009,0.879103,0.002954
2010,1.052608,0.001261
2011,0.806605,0.001514


## 10.4 透视表和交叉表

In [271]:
pd.pivot_table??
#pandas的顶级函数，margins可以分项小计
#pd.pivot_table(
#    data,
#    values=None,
#    index=None,
#    columns=None,
#    aggfunc='mean',
#    fill_value=None,
#    margins=False,
#    dropna=True,
#    margins_name='All',
#)

In [249]:
tips[:10]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [247]:
tips.pivot_table(index=['day', 'smoker'])
#使用index=['day', 'smoker']参数，指定根据day列和smoker列分组计算平均数
#index的参数值出现在结果透视表的行——作为行索引？
#aggfunc参数默认是均值
#分组的项day和smoker作为行的层次索引出现

size       tip   tip_pct  total_bill
day  smoker                                          
Fri  No      2.250000  2.812500  0.151650   18.420000
     Yes     2.066667  2.714000  0.174783   16.813333
Sat  No      2.555556  3.102889  0.158048   19.661778
     Yes     2.476190  2.875476  0.147906   21.276667
Sun  No      2.929825  3.167895  0.160113   20.506667
     Yes     2.578947  3.516842  0.187250   24.120000
Thur No      2.488889  2.673778  0.160298   17.113111
     Yes     2.352941  3.030000  0.163863   19.190588

In [253]:
tips.pivot_table(index=['day', 'smoker'],aggfunc='count')#修改aggfunc为count

size  time  tip  tip_pct  total_bill
day  smoker                                      
Fri  No         4     4    4        4           4
     Yes       15    15   15       15          15
Sat  No        45    45   45       45          45
     Yes       42    42   42       42          42
Sun  No        57    57   57       57          57
     Yes       19    19   19       19          19
Thur No        45    45   45       45          45
     Yes       17    17   17       17          17

In [254]:
tips.pivot_table(index=['day', 'smoker'],aggfunc='describe')#修改aggfunc为describe

size                                                   tip         \
             25%  50%   75% count  max      mean  min       std    25%    50%   
day  smoker                                                                     
Fri  No      2.0  2.0  2.25   4.0  3.0  2.250000  2.0  0.500000  2.625  3.125   
     Yes     2.0  2.0  2.00  15.0  4.0  2.066667  1.0  0.593617  1.960  2.500   
Sat  No      2.0  2.0  3.00  45.0  4.0  2.555556  1.0  0.784960  2.010  2.750   
     Yes     2.0  2.0  3.00  42.0  5.0  2.476190  1.0  0.862161  2.000  2.690   
Sun  No      2.0  3.0  4.00  57.0  6.0  2.929825  2.0  1.032674  2.000  3.020   
     Yes     2.0  2.0  3.00  19.0  5.0  2.578947  2.0  0.901591  3.000  3.500   
Thur No      2.0  2.0  2.00  45.0  6.0  2.488889  1.0  1.179796  1.800  2.180   
     Yes     2.0  2.0  2.00  17.0  4.0  2.352941  2.0  0.701888  2.000  2.560   

             ...   tip_pct           total_bill                                \
             ...       min       std        25%     50%      75% count    max   
day  smoker  ...                                                                
Fri  No      ...  0.120385  0.028123     15.100  19.235  22.5550   4.0  22.75   
     Yes     ...  0.103555  0.051293     11.690  13.420  18.6650  15.0  40.17   
Sat  No      ...  0.056797  0.039767     14.730  17.820  20.6500  45.0  48.33   
     Yes     ...  0.035638  0.061375     13.405  20.390  26.7925  42.0  50.81   
Sun  No      ...  0.059447  0.042347     14.780  18.430  25.0000  57.0  48.17   
     Yes     ...  0.065660  0.154134     17.165  23.100  32.3750  19.0  45.35   
Thur No      ...  0.072961  0.038774     11.690  15.950  20.2700  45.0  41.19   
     Yes     ...  0.090014  0.039389     13.510  16.470  19.8100  17.0  43.11   

                                          
                  mean    min        std  
day  smoker                               
Fri  No      18.420000  12.46   5.059282  
     Yes     16.813333   5.75   9.086388  
Sat  No      19.661778   7.25   8.939181  
     Yes     21.276667   3.07  10.069138  
Sun  No      20.506667   8.77   8.130189  
     Yes     24.120000   7.25  10.442511  
Thur No      17.113111   7.51   7.721728  
     Yes     19.190588  10.34   8.355149  

[8 rows x 32 columns]

In [255]:
tips.pivot_table(index=['day', 'smoker'],aggfunc='describe').T#修改aggfunc为describe,并转置

day                     Fri                   Sat                   Sun  \
smoker                   No        Yes         No        Yes         No   
size       25%     2.000000   2.000000   2.000000   2.000000   2.000000   
           50%     2.000000   2.000000   2.000000   2.000000   3.000000   
           75%     2.250000   2.000000   3.000000   3.000000   4.000000   
           count   4.000000  15.000000  45.000000  42.000000  57.000000   
           max     3.000000   4.000000   4.000000   5.000000   6.000000   
           mean    2.250000   2.066667   2.555556   2.476190   2.929825   
           min     2.000000   1.000000   1.000000   1.000000   2.000000   
           std     0.500000   0.593617   0.784960   0.862161   1.032674   
tip        25%     2.625000   1.960000   2.010000   2.000000   2.000000   
           50%     3.125000   2.500000   2.750000   2.690000   3.020000   
           75%     3.312500   3.240000   3.390000   3.197500   3.920000   
           count   4.000000  15.000000  45.000000  42.000000  57.000000   
           max     3.500000   4.730000   9.000000  10.000000   6.000000   
           mean    2.812500   2.714000   3.102889   2.875476   3.167895   
           min     1.500000   1.000000   1.000000   1.000000   1.010000   
           std     0.898494   1.077668   1.642088   1.630580   1.224785   
tip_pct    25%     0.137239   0.133739   0.136240   0.091797   0.139780   
           50%     0.149241   0.173913   0.150152   0.153624   0.161665   
           75%     0.163652   0.209240   0.183915   0.190502   0.185185   
           count   4.000000  15.000000  45.000000  42.000000  57.000000   
           max     0.187735   0.263480   0.291990   0.325733   0.252672   
           mean    0.151650   0.174783   0.158048   0.147906   0.160113   
           min     0.120385   0.103555   0.056797   0.035638   0.059447   
           std     0.028123   0.051293   0.039767   0.061375   0.042347   
total_bill 25%    15.100000  11.690000  14.730000  13.405000  14.780000   
           50%    19.235000  13.420000  17.820000  20.390000  18.430000   
           75%    22.555000  18.665000  20.650000  26.792500  25.000000   
           count   4.000000  15.000000  45.000000  42.000000  57.000000   
           max    22.750000  40.170000  48.330000  50.810000  48.170000   
           mean   18.420000  16.813333  19.661778  21.276667  20.506667   
           min    12.460000   5.750000   7.250000   3.070000   8.770000   
           std     5.059282   9.086388   8.939181  10.069138   8.130189   

day                               Thur             
smoker                  Yes         No        Yes  
size       25%     2.000000   2.000000   2.000000  
           50%     2.000000   2.000000   2.000000  
           75%     3.000000   2.000000   2.000000  
           count  19.000000  45.000000  17.000000  
           max     5.000000   6.000000   4.000000  
           mean    2.578947   2.488889   2.352941  
           min     2.000000   1.000000   2.000000  
           std     0.901591   1.179796   0.701888  
tip        25%     3.000000   1.800000   2.000000  
           50%     3.500000   2.180000   2.560000  
           75%     4.000000   3.000000   4.000000  
           count  19.000000  45.000000  17.000000  
           max     6.500000   6.700000   5.000000  
           mean    3.516842   2.673778   3.030000  
           min     1.500000   1.250000   2.000000  
           std     1.261151   1.282964   1.113491  
tip_pct    25%     0.097723   0.137741   0.148038  
           50%     0.138122   0.153492   0.153846  
           75%     0.215325   0.184843   0.194837  
           count  19.000000  45.000000  17.000000  
           max     0.710345   0.266312   0.241255  
           mean    0.187250   0.160298   0.163863  
           min     0.065660   0.072961   0.090014  
           std     0.154134   0.038774   0.039389  
total_bill 25%    17.165000  11.690000  13.510000  
           50%    23.100000  15.950000  16.47

In [248]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],columns='smoker')
#如果只聚合tip_pct和size，并将这两项根据time和day进行分组，计算默认的均值，并将smoker放到列显示，day放到行显示
#columns指定分组的键，和index的区别是，columns出现在结果的列（作为列名出现）

size             tip_pct          
smoker             No       Yes        No       Yes
time   day                                         
Dinner Fri   2.000000  2.222222  0.139622  0.165347
       Sat   2.555556  2.476190  0.158048  0.147906
       Sun   2.929825  2.578947  0.160113  0.187250
       Thur  2.000000       NaN  0.159744       NaN
Lunch  Fri   3.000000  1.833333  0.187735  0.188937
       Thur  2.500000  2.352941  0.160311  0.163863

In [267]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'])
#如果不指定columns='smoker',则不在列上按smoker分组，当然smoker也不会作为列名出现

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301

In [269]:
tips.pivot_table(['tip','tip_pct', 'size','smoker'], index=['time', 'day'])
#增加计算均值的项目tip，smoker虽然也加进去了，但其值为no和yes，没有均值，没显示??

size       tip   tip_pct
time   day                               
Dinner Fri   2.166667  2.940000  0.158916
       Sat   2.517241  2.993103  0.153152
       Sun   2.842105  3.255132  0.166897
       Thur  2.000000  3.000000  0.159744
Lunch  Fri   2.000000  2.382857  0.188765
       Thur  2.459016  2.767705  0.161301

In [262]:
tips[:10]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240
6,8.77,2.00,No,Sun,Dinner,2,0.228050
7,26.88,3.12,No,Sun,Dinner,4,0.116071
8,15.04,1.96,No,Sun,Dinner,2,0.130319
9,14.78,3.23,No,Sun,Dinner,2,0.218539


In [250]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'], columns='smoker', margins=True)
#margins=True增加按列分组小计和按行分组小计（合计）
#其值对应于单个等级中所有数据的分组统计，行只有一个索引，因此是合计
#由于colums='smoker'是次级索引，所以分组小计是在次级索引上进行的
#smoker-all，表示不考虑是否吸烟的平均值

size                       tip_pct                    
smoker             No       Yes       All        No       Yes       All
time   day                                                             
Dinner Fri   2.000000  2.222222  2.166667  0.139622  0.165347  0.158916
       Sat   2.555556  2.476190  2.517241  0.158048  0.147906  0.153152
       Sun   2.929825  2.578947  2.842105  0.160113  0.187250  0.166897
       Thur  2.000000       NaN  2.000000  0.159744       NaN  0.159744
Lunch  Fri   3.000000  1.833333  2.000000  0.187735  0.188937  0.188765
       Thur  2.500000  2.352941  2.459016  0.160311  0.163863  0.161301
All          2.668874  2.408602  2.569672  0.159328  0.163196  0.160803

In [272]:
tips.pivot_table(['tip_pct', 'size'], index=['time', 'day'],  margins=True)
#没有指定列，margins=True只按行小计（也是总计）

size   tip_pct
time   day                     
Dinner Fri   2.166667  0.158916
       Sat   2.517241  0.153152
       Sun   2.842105  0.166897
       Thur  2.000000  0.159744
Lunch  Fri   2.000000  0.188765
       Thur  2.459016  0.161301
All          2.569672  0.160803

In [273]:
tips.pivot_table(['tip','tip_pct', 'size'], index=['time', 'day'],  margins=True)

size       tip   tip_pct
time   day                               
Dinner Fri   2.166667  2.940000  0.158916
       Sat   2.517241  2.993103  0.153152
       Sun   2.842105  3.255132  0.166897
       Thur  2.000000  3.000000  0.159744
Lunch  Fri   2.000000  2.382857  0.188765
       Thur  2.459016  2.767705  0.161301
All          2.569672  2.998279  0.160803

In [251]:
tips.pivot_table('tip_pct', index=['time', 'smoker'], columns='day', aggfunc=len, margins=True)
#传递其他聚合函数，只需把函数名传递给aggfunc

day             Fri   Sat   Sun  Thur    All
time   smoker                               
Dinner No       3.0  45.0  57.0   1.0  106.0
       Yes      9.0  42.0  19.0   NaN   70.0
Lunch  No       1.0   NaN   NaN  44.0   45.0
       Yes      6.0   NaN   NaN  17.0   23.0
All            19.0  87.0  76.0  62.0  244.0

In [252]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean', fill_value=0)
#指定缺失值的填充值为0

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No      0.000000  0.137931  0.000000  0.000000
            Yes     0.000000  0.325733  0.000000  0.000000
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893  0.000000
       3    No      0.000000  0.154661  0.152663  0.000000
            Yes     0.000000  0.144995  0.152660  0.000000
       4    No      0.000000  0.150096  0.148143  0.000000
            Yes     0.117750  0.124515  0.193370  0.000000
       5    No      0.000000  0.000000  0.206928  0.000000
            Yes     0.000000  0.106572  0.065660  0.000000
       6    No      0.000000  0.000000  0.103799  0.000000
Lunch  1    No      0.000000  0.000000  0.000000  0.181728
            Yes     0.223776  0.000000  0.000000  0.000000
       2    No      0.000000  0.000000  0.000000  0.166005
            Yes     0.181969  0.000000  0.000000  0.158843
       3    No      0.187735  0.000000  0.000000  0.084246
            Yes     0.000000  0.000000  0.000000  0.204952
       4    No      0.000000  0.000000  0.000000  0.138919
            Yes     0.000000  0.000000  0.000000  0.155410
       5    No      0.000000  0.000000  0.000000  0.121389
       6    No      0.000000  0.000000  0.000000  0.173706

In [274]:
tips.pivot_table('tip_pct', index=['time', 'size', 'smoker'], columns='day', aggfunc='mean')
#作为对比，不指定缺失值的填充值

day                      Fri       Sat       Sun      Thur
time   size smoker                                        
Dinner 1    No           NaN  0.137931       NaN       NaN
            Yes          NaN  0.325733       NaN       NaN
       2    No      0.139622  0.162705  0.168859  0.159744
            Yes     0.171297  0.148668  0.207893       NaN
       3    No           NaN  0.154661  0.152663       NaN
            Yes          NaN  0.144995  0.152660       NaN
       4    No           NaN  0.150096  0.148143       NaN
            Yes     0.117750  0.124515  0.193370       NaN
       5    No           NaN       NaN  0.206928       NaN
            Yes          NaN  0.106572  0.065660       NaN
       6    No           NaN       NaN  0.103799       NaN
Lunch  1    No           NaN       NaN       NaN  0.181728
            Yes     0.223776       NaN       NaN       NaN
       2    No           NaN       NaN       NaN  0.166005
            Yes     0.181969       NaN       NaN  0.158843
       3    No      0.187735       NaN       NaN  0.084246
            Yes          NaN       NaN       NaN  0.204952
       4    No           NaN       NaN       NaN  0.138919
            Yes          NaN       NaN       NaN  0.155410
       5    No           NaN       NaN       NaN  0.121389
       6    No           NaN       NaN       NaN  0.173706

### 交叉表：crosstab

In [281]:
data=pd.DataFrame(pd.read_csv('examples/10.4.csv'))

In [282]:
data

,Sample,Nationality,Handedness
0,1,USA,Right-handed
1,2,Japan,Left-handed
2,3,USA,Right-handed
3,4,Japan,Right-handed
4,5,Japan,Left-handed
5,6,Japan,Right-handed
6,7,USA,Right-handed
7,8,USA,Left-handed
8,9,Japan,Right-handed
9,10,USA,Right-handed


In [283]:
pd.crosstab(data.Nationality, data.Handedness, margins=True)
#使用crosstab函数统计选手的国籍和用手习惯
#crosstab前两个参数传递，可以是数组或Series，或数组列表

Handedness,Left-handed,Right-handed,All
Nationality,,,
Japan,2,3,5
USA,1,4,5
All,3,7,10


In [294]:
#使用pivot_table实现上述统计
data.pivot_table(index=['Nationality'],columns='Handedness',aggfunc='count',margins=True)
#国别出现在行，index来指定
#用手习惯出现在列，columns来指定
#需要小计，margins=True
#聚合函数是计数，而非求默认的求均值，需要制定aggfunc为count

Sample                 
Handedness  Left-handed Right-handed All
Nationality                             
Japan                 2            3   5
USA                   1            4   5
All                   3            7  10

In [284]:
pd.crosstab([tips.time, tips.day], tips.smoker, margins=True)
#传递数组列表作为cross_tab的参数，最终的DF数据，行是有层次索引的

smoker        No  Yes  All
time   day                
Dinner Fri     3    9   12
       Sat    45   42   87
       Sun    57   19   76
       Thur    1    0    1
Lunch  Fri     1    6    7
       Thur   44   17   61
All          151   93  244